In [1]:
from transformers import LiltForPretraining, HarrysConfig, HarrysInvConfig
from pl_lilt_datamodule import LiltPretrainingDataModule
import argparse

In [2]:
parser = argparse.ArgumentParser()
parser = LiltPretrainingDataModule.add_model_specific_args(parser)
args = parser.parse_args('')

dm = LiltPretrainingDataModule(args)
dm.prepare_data()
dm.setup()
train_loader = dm.train_dataloader()
batch = next(iter(train_loader))

/home/hj36wegi/.conda/envs/adapters/lib/python3.11/site-packages/transformers/models/layoutlmv2/feature_extraction_layoutlmv2.py:30: FutureWarning: The class LayoutLMv2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use LayoutLMv2ImageProcessor instead.
  warnings.warn(


In [3]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'bbox', 'bbox_loc', 'input_labels', 'bbox_labels'])

In [4]:
lang_adapter_config = HarrysInvConfig(text_output_adapter=True, layout_output_adapter=False)
layout_adapter_config = HarrysConfig(text_output_adapter=False, layout_output_adapter=True)
task_adapter_config = HarrysConfig(text_output_adapter=True, layout_output_adapter=False)

In [5]:
model = LiltForPretraining.from_pretrained("nielsr/lilt-xlm-roberta-base")

Some weights of LiltForPretraining were not initialized from the model checkpoint at nielsr/lilt-xlm-roberta-base and are newly initialized: ['lm_head.cai_head.bias', 'lm_head.kpl_head.dense.weight', 'lm_head.kpl_head.layer_norm.weight', 'lm_head.kpl_head.layer_norm.bias', 'lm_head.cai_head.dense.bias', 'lm_head.mvlm_head.layer_norm.weight', 'lm_head.mvlm_head.dense.bias', 'lm_head.cai_head.decoder.bias', 'lm_head.cai_head.dense.weight', 'lm_head.kpl_head.bias', 'lm_head.cai_head.layer_norm.weight', 'lm_head.mvlm_head.bias', 'lm_head.mvlm_head.dense.weight', 'lm_head.kpl_head.decoder.weight', 'lm_head.cai_head.decoder.weight', 'lm_head.kpl_head.decoder.bias', 'lm_head.mvlm_head.layer_norm.bias', 'lm_head.kpl_head.dense.bias', 'lm_head.cai_head.layer_norm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model.add_adapter("layout", layout_adapter_config)
model.add_adapter("language", lang_adapter_config)
model.add_adapter("task", task_adapter_config)

In [24]:
model.train_adapter(["layout","task", "language"])
# model.freeze_model(False)


In [21]:
out = model(**batch)

language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout
language
task
layout


In [22]:
for n,p in model.lilt.encoder.layer[0].named_parameters():
    if 'adapter' in n:
        if p.requires_grad:
            print("+ " + n)
        else:
            print("- " + n)

+ output.adapters.language.adapter_down.0.weight
+ output.adapters.language.adapter_down.0.bias
+ output.adapters.language.adapter_up.weight
+ output.adapters.language.adapter_up.bias
+ output.adapters.task.adapter_down.0.weight
+ output.adapters.task.adapter_down.0.bias
+ output.adapters.task.adapter_up.weight
+ output.adapters.task.adapter_up.bias
+ layout_output.adapters.layout.adapter_down.0.weight
+ layout_output.adapters.layout.adapter_down.0.bias
+ layout_output.adapters.layout.adapter_up.weight
+ layout_output.adapters.layout.adapter_up.bias


In [23]:
p

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)